In [ ]:
# Import required libraries
!pip install ipywidgets
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Load the dataset
laptop_data = pd.read_csv('laptop_data.csv')

In [ ]:
laptop_data.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [ ]:
# Display the first few rows of the dataset to understand the structure
print("Laptop Data Overview:")
print(laptop_data.head())

Laptop Data Overview:
    Brand   Model Name Core CPU Manufacturer Clock Speed RAM Size  \
0    Dell     Inspiron   i5            Intel     2.4 GHz      8GB   
1     MSI         GL65   i7            Intel     2.6 GHz     16GB   
2      HP    EliteBook   i7            Intel     2.8 GHz     16GB   
3  Lenovo      IdeaPad   i3            Intel     2.1 GHz      8GB   
4    ASUS  ZenBook Pro   i9            Intel     3.1 GHz     64GB   

  Storage Type Display Type Display Size Graphics Processor Screen Resolution  \
0          SSD          LCD        15.6"          Intel UHD         1920x1080   
1      HDD+SSD          IPS        15.6"         NVIDIA GTX         1920x1080   
2          SSD          LED          14"          Intel UHD         1920x1080   
3          HDD           TN        15.6"          Intel UHD          1366x768   
4          SSD         OLED        15.6"         NVIDIA RTX         3840x2160   

           OS Laptop Weight    Special Features Warranty Average Battery Lif

In [ ]:
# Basic statistics to understand numerical columns
print("\nLaptop Data Statistics:")
print(laptop_data.describe())


Laptop Data Statistics:
       Brand Model Name Core CPU Manufacturer Clock Speed RAM Size  \
count     20         20   20               20          20       20   
unique     9         20    8                3          10        5   
top     Dell   Inspiron   i7            Intel     2.8 GHz     16GB   
freq       3          1    8               17           5       11   

       Storage Type Display Type Display Size Graphics Processor  \
count            20           20           20                 20   
unique            3            8            6                  8   
top             SSD          IPS        15.6"          Intel UHD   
freq             17            9            9                  5   

       Screen Resolution          OS Laptop Weight  Special Features Warranty  \
count                 20          20            20                20       20   
unique                 7           5            18                19        3   
top            1920x1080  Windows 10    

In [ ]:
# Check for missing values
print("\nMissing Values in Laptop Data:")
print(laptop_data.isnull().sum())


Missing Values in Laptop Data:
Brand                   0
Model Name              0
Core                    0
CPU Manufacturer        0
Clock Speed             0
RAM Size                0
Storage Type            0
Display Type            0
Display Size            0
Graphics Processor      0
Screen Resolution       0
OS                      0
Laptop Weight           0
Special Features        0
Warranty                0
Average Battery Life    0
Price                   0
Description             0
dtype: int64


In [ ]:
print(laptop_data.dtypes)

Brand                   object
Model Name              object
Core                    object
CPU Manufacturer        object
Clock Speed             object
RAM Size                object
Storage Type            object
Display Type            object
Display Size            object
Graphics Processor      object
Screen Resolution       object
OS                      object
Laptop Weight           object
Special Features        object
Warranty                object
Average Battery Life    object
Price                   object
Description             object
dtype: object


In [ ]:
# Fill missing values with an empty string to avoid issues during tokenization
laptop_data.fillna('', inplace=True)

In [ ]:
# Combine the brand and description into a single text column for tokenization
laptop_data['text'] = laptop_data['Brand'] + ' ' + laptop_data['Model Name'] + ' ' +  laptop_data['Description']

In [ ]:
# Check the new 'text' column
print("\nLaptop Data Text Feature:")
print(laptop_data['text'].head())


Laptop Data Text Feature:
0    Dell Inspiron The Dell Inspiron is a versatile...
1    MSI GL65 The MSI GL65 is a high-performance la...
2    HP EliteBook The HP EliteBook is a premium lap...
3    Lenovo IdeaPad The Lenovo IdeaPad is a versati...
4    ASUS ZenBook Pro The ASUS ZenBook Pro is a hig...
Name: text, dtype: object


## Feature Engineering with Hugging Face LLM

In [ ]:
# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Function to create embeddings for the text data
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

In [ ]:
# Apply embedding generation to the dataset
laptop_data['embeddings'] = laptop_data['text'].apply(lambda x: get_embeddings(x))

In [ ]:
# Check the embeddings for the first product
print("\nLaptop Data Embeddings (first product):")
print(laptop_data['embeddings'].iloc[0])



Laptop Data Embeddings (first product):
[[-1.15857989e-01  1.64844379e-01 -2.00943891e-02  6.97854161e-02
   9.24687833e-02 -1.13538936e-01  2.84679537e-03  1.49879307e-01
   1.12662531e-01  2.06302553e-02  1.04948461e-01 -6.58793561e-03
   3.27110477e-03  7.23243803e-02  8.80477130e-02  6.34213239e-02
   1.76339149e-01 -2.23702937e-01  2.61930656e-02 -1.10580720e-01
  -1.28620118e-02 -5.88487722e-02 -1.52060211e-01  6.85454756e-02
   1.07151516e-01  1.07430629e-02  4.56405655e-02  9.47704073e-04
  -2.11198464e-01 -2.31197011e-02 -7.13143125e-02  8.36236775e-03
  -7.97853153e-03 -7.89451078e-02 -1.19757786e-01  2.79038921e-02
   1.15992635e-01 -8.35279524e-02  4.74919304e-02 -2.21115559e-01
  -2.75090914e-02 -2.55886950e-02  1.24985032e-01  5.36295176e-02
   1.76145852e-01 -1.94689080e-01 -8.08203444e-02  8.98369402e-02
  -7.68470466e-02 -3.82472351e-02 -8.62914622e-02  1.30736277e-01
  -4.54680584e-02  1.14184596e-01 -1.11450270e-01  9.27258283e-02
   9.50029641e-02 -9.73976869e-03 -

## Model Selection (Cosine Similarity)

In [ ]:
# Compute cosine similarity between products in the dataset
laptop_similarity = cosine_similarity(np.vstack(laptop_data['embeddings'].values))

## Function to get recommendations based on similarity

In [ ]:
# Function to find the index of the product by name or brand
def find_product_index(data, product_name=None, brand_name=None):
    if product_name:
        # Search for a product that contains the product name (case-insensitive)
        product_idx = data[data['text'].str.contains(product_name, case=False, na=False)].index
    elif brand_name:
        # Search for a product that contains the brand name (case-insensitive)
        product_idx = data[data['Brand'].str.contains(brand_name, case=False, na=False)].index
    else:
        # If neither is provided, return None
        return None

    if len(product_idx) == 0:
        print(f"No matching product found for name: {product_name} or brand: {brand_name}")
        return None

    # Return the first matching index
    return product_idx[0]

In [ ]:
# Function to recommend products based on product name or brand
def recommend_products_by_name(similarity_matrix, data, product_name=None, brand_name=None, top_n=5):
    # Find the index of the product by name or brand
    product_idx = find_product_index(data, product_name=product_name, brand_name=brand_name)

    if product_idx is None:
        print("No product found with the given name or brand.")
        return None

    # Get similarity scores for the selected product
    similarity_scores = similarity_matrix[product_idx]

    # Sort the scores in descending order and return the top N recommendations
    similar_products = similarity_scores.argsort()[-top_n-1:-1][::-1]

    return data.iloc[similar_products]


## Testing the Recommendation System

### Usage to recommend laptops by product name or brand

In [ ]:
# Recommend based on product name
product_name_example = "HP Pavilion"
recommended_laptops_by_name = recommend_products_by_name(laptop_similarity, laptop_data, product_name=product_name_example)

# Display recommended laptops based on product name
if recommended_laptops_by_name is not None:
    print("\nRecommended Laptops based on product name '{}':".format(product_name_example))
    print(recommended_laptops_by_name[['Brand', 'Description']])



Recommended Laptops based on product name 'HP Pavilion':
   Brand                                        Description
2     HP  The HP EliteBook is a premium laptop designed ...
17    HP  The HP ENVY x360 is a versatile 2-in-1 convert...
0   Dell  The Dell Inspiron is a versatile laptop that c...
14   MSI  The MSI Prestige 14 is a compact and stylish l...
10  Dell  The Dell XPS 15 is a premium laptop that combi...


In [ ]:
# Recommend based on brand name
brand_name_example = "Dell"
recommended_laptops_by_brand = recommend_products_by_name(laptop_similarity, laptop_data, brand_name=brand_name_example)

# Display recommended laptops based on brand name
if recommended_laptops_by_brand is not None:
    print("\nRecommended Laptops based on brand '{}':".format(brand_name_example))
    print(recommended_laptops_by_brand[['Brand', 'Description']])


Recommended Laptops based on brand 'Dell':
   Brand                                        Description
10  Dell  The Dell XPS 15 is a premium laptop that combi...
8     HP  The HP Pavilion is a budget-friendly laptop th...
16  Dell  The Dell Precision 5550 is a high-performance ...
2     HP  The HP EliteBook is a premium laptop designed ...
17    HP  The HP ENVY x360 is a versatile 2-in-1 convert...


## Interactive Widgets with ipywidgets

In [ ]:
# Create widgets
input_type = widgets.Dropdown(
    options=['name', 'brand'],
    description='Search by:',
    style={'description_width': 'initial'}
)

text_input = widgets.Text(
    value='',
    placeholder='Enter the product name or brand...',
    description='Input:',
    style={'description_width': 'initial'}
)

output = widgets.Output()


In [ ]:
# Define the function that will be triggered by widget interactions
def on_button_click(b):
    with output:
        output.clear_output()  # Clear previous output
        user_input_type = input_type.value
        user_input_text = text_input.value

        if user_input_type == 'name':
            recommended_laptops = recommend_products_by_name(laptop_similarity, laptop_data, product_name=user_input_text)
        elif user_input_type == 'brand':
            recommended_laptops = recommend_products_by_name(laptop_similarity, laptop_data, brand_name=user_input_text)

        if recommended_laptops is not None:
            print(f"\nRecommended Laptops based on {user_input_type} '{user_input_text}':")
            display(recommended_laptops[['Brand', 'Description']])
        else:
            print(f"No recommendations found for {user_input_type} '{user_input_text}'.")


In [ ]:
# Button to trigger the recommendation process
button = widgets.Button(
    description='Get Recommendations',
    button_style='success',  # 'success', 'info', 'warning', 'danger'
    tooltip='Click to get recommendations'
)

button.on_click(on_button_click)

# Display the widgets and the output area
display(input_type, text_input, button, output)


Dropdown(description='Search by:', options=('name', 'brand'), style=DescriptionStyle(description_width='initia…

Text(value='', description='Input:', placeholder='Enter the product name or brand...', style=DescriptionStyle(…

Button(button_style='success', description='Get Recommendations', style=ButtonStyle(), tooltip='Click to get r…

Output()